In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, month, countDistinct, lag, col, sum, when
from pyspark.sql.window import Window

# create a SparkSession
spark = SparkSession.builder.appName("myApp").getOrCreate()
spark

In [4]:
# read the CSV file into a DataFrame
df = spark.read.csv("../pret", header=True, inferSchema=True)


In [6]:

# filter the DataFrame to only include new clients (i.e., those who did not appear in any previous month)
windowSpec = Window.partitionBy("code_agent", "code_client").orderBy("date_de_deboursement")
df_new_clients = df.withColumn("prev_code_client", lag("code_client").over(windowSpec)) \
                  .withColumn("is_new_client", when(col("prev_code_client").isNull(), 1).otherwise(0)) \
                  .filter(col("is_new_client") == 1) \
                  .drop("prev_code_client", "is_new_client")

# group by code_agent and year/month and count distinct code_client
num_clients_df = df_new_clients.groupBy("code_agent", year("date_de_deboursement").alias("year"), month("date_de_deboursement").alias("month")).agg(countDistinct("code_client").alias("num_new_clients"))

# add a column with the previous month's number of clients for each code_agent
windowSpec = Window.partitionBy("code_agent").orderBy("year", "month")
lagged_df = num_clients_df.withColumn("prev_num_new_clients", lag("num_new_clients").over(windowSpec))

# calculate the percentage of new clients relative to the total number of clients for each code_agent
final_df = lagged_df.withColumn("num_all_clients", sum("num_new_clients").over(windowSpec)) \
                   .withColumn("pct_new_clients", col("num_new_clients") / col("num_all_clients") * 100)

# show the final DataFrame
final_df.show(10)

+----------+----+-----+---------------+--------------------+---------------+------------------+
|code_agent|year|month|num_new_clients|prev_num_new_clients|num_all_clients|   pct_new_clients|
+----------+----+-----+---------------+--------------------+---------------+------------------+
|      2001|2019|    3|              2|                null|              2|             100.0|
|      2001|2019|    5|              2|                   2|              4|              50.0|
|      2002|2019|    2|             19|                null|             19|             100.0|
|      2002|2019|    3|             15|                  19|             34| 44.11764705882353|
|      2002|2019|    4|             31|                  15|             65| 47.69230769230769|
|      2002|2019|    5|             32|                  31|             97| 32.98969072164948|
|      2002|2019|    6|             24|                  32|            121|19.834710743801654|
|      2002|2019|    7|             39| 

In [9]:
# Group by code_agent and calculate the mean percentage of all months
mean_pct_df = final_df.groupBy("code_agent").agg({"pct_new_clients": "mean"})

# Show the resulting DataFrame
mean_pct_df.tail(400)

[Row(code_agent=3270, avg(pct_new_clients)=9.137631915823214),
 Row(code_agent=3272, avg(pct_new_clients)=8.938538271290975),
 Row(code_agent=3273, avg(pct_new_clients)=8.85185122589705),
 Row(code_agent=3274, avg(pct_new_clients)=9.374524447461322),
 Row(code_agent=3275, avg(pct_new_clients)=9.240737963284175),
 Row(code_agent=3276, avg(pct_new_clients)=8.252254644841436),
 Row(code_agent=3277, avg(pct_new_clients)=9.410548179803664),
 Row(code_agent=3278, avg(pct_new_clients)=8.55950436581708),
 Row(code_agent=3279, avg(pct_new_clients)=9.665168453417495),
 Row(code_agent=3281, avg(pct_new_clients)=9.30251268052417),
 Row(code_agent=3282, avg(pct_new_clients)=9.499918855289394),
 Row(code_agent=3286, avg(pct_new_clients)=8.575668057277179),
 Row(code_agent=3287, avg(pct_new_clients)=9.164846685529392),
 Row(code_agent=3288, avg(pct_new_clients)=61.11111111111111),
 Row(code_agent=3291, avg(pct_new_clients)=8.668132405457115),
 Row(code_agent=3292, avg(pct_new_clients)=75.0),
 Row(cod